In [1]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import os
import re
import html2text
import pickle
from sec_edgar_downloader import Downloader

In [2]:
# Get the most recent filing
f = open("sec-edgar-filings/AAL/10-K/0001193125-15-061145/filing-details.html", "r")
raw_10k = f.read()

In [3]:
soup = BeautifulSoup(raw_10k, 'lxml')

In [4]:
res = list(soup.find_all('b'))
res = [str(x) for x in res]
res_str = '\n'.join(res)

In [5]:
matches = re.compile(r'(item\s(7[\.\s]|8(A|)[\.\s])|'
                             'discussion\sand\sanalysis\sof\s(consolidated\sfinancial|financial)\scondition|'
                             '(consolidated\sfinancial|financial)\sstatements\sand\ssupplementary\sdata)', re.IGNORECASE)

In [6]:
matches_array = pd.DataFrame([(match.group(), match.start()) for match in matches.finditer(res_str)])
matches_array.head()

,0,1
0,ITEM 7.,13992
1,DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION,14013
2,ITEM 8A.,21020
3,CONSOLIDATED FINANCIAL STATEMENTS AND SUPPLEME...,21029
4,CONSOLIDATED FINANCIAL STATEMENTS AND SUPPLEME...,42175


In [7]:
matches = re.compile(r'ITEM', re.IGNORECASE)

In [8]:
matches_array = pd.DataFrame([(match.group(), match.start()) for match in matches.finditer(raw_10k)])
matches_array.head()

,0,1
0,Item,18637
1,Item,30456
2,Item,31217
3,Item,31960
4,Item,32722


In [9]:
raw_10k[:5000]

'<html><body><document>\n<type>10-K\n<sequence>1\n<filename>d829913d10k.htm\n<description>FORM 10-K\n<text>\n<title>Form 10-K</title>\n<h5 align="left"><a href="#toc">Table of Contents</a></h5>\n<p style="line-height:4px;margin-top:0px;margin-bottom:0px;border-bottom:2pt solid #000000">&#160;</p>\n<p style="line-height:3px;margin-top:0px;margin-bottom:2px;border-bottom:0.5pt solid #000000">&#160;</p> <p align="center" style="margin-top:1px;margin-bottom:0px"><font size="2" style="font-family:Times New Roman"><b>UNITED STATES SECURITIES AND\nEXCHANGE COMMISSION </b></font></p> <p align="center" style="margin-top:0px;margin-bottom:0px"><font size="2" style="font-family:Times New Roman"><b>Washington, D.C. 20549 </b></font></p>\n<p style="font-size:1px;margin-top:0px;margin-bottom:0px">&#160;</p><center> <p style="line-height:6px;margin-top:0px;margin-bottom:2px;border-bottom:1pt solid #000000;width:21%">&#160;</p></center>\n<p align="center" style="margin-top:1px;margin-bottom:0px"><font

In [10]:
matches = re.finditer(r'(item.{20}(7[\.\s]|8(A|)[\.\s])|'
                             'discussion\sand\sanalysis\sof\s(consolidated\sfinancial|financial)\scondition|'
                             '(consolidated\sfinancial|financial)\sstatements\sand\ssupplementary\sdata)', raw_10k, re.IGNORECASE)
locations = [x for x in matches]
locations

[<re.Match object; span=(37293, 37339), match='Discussion and Analysis of Financial Condition'>,
 <re.Match object; span=(38888, 38944), match='Consolidated Financial Statements and Supplementa>,
 <re.Match object; span=(39715, 39771), match='Consolidated Financial Statements and Supplementa>,
 <re.Match object; span=(58563, 58609), match='Discussion and Analysis of Financial Condition'>,
 <re.Match object; span=(68365, 68411), match='Discussion and Analysis of Financial Condition'>,
 <re.Match object; span=(70730, 70776), match='Discussion and Analysis of Financial Condition'>,
 <re.Match object; span=(70893, 70939), match='Discussion and Analysis of Financial Condition'>,
 <re.Match object; span=(74080, 74126), match='Discussion and Analysis of Financial Condition'>,
 <re.Match object; span=(218373, 218419), match='Discussion and Analysis of Financial Condition'>,
 <re.Match object; span=(315947, 315993), match='Discussion and Analysis of Financial Condition'>,
 <re.Match object; spa

In [11]:
matches = re.finditer('..>ITEM.{1,10}7[^a-zA-Z]{1,10}management', raw_10k, re.IGNORECASE)
locations = [x for x in matches]
locations[:3]

[<re.Match object; span=(723646, 723672), match='/a>ITEM&#160;7. MANAGEMENT'>]

In [12]:
matches = re.finditer('..>ITEM.{1,10}8(|A)[^a-zA-Z]{1,10}(consolidated|financial)', raw_10k, re.IGNORECASE)
locations = [x for x in matches]
locations[:3]

[<re.Match object; span=(1614421, 1614455), match='/a>ITEM&#160;8A.&#160;CONSOLIDATED'>]

In [13]:
# Check each filing to make sure there's a match
for filing in os.listdir("sec-edgar-filings/AAL/10-K/"):
    
    locs = []
    
    if filing == '.DS_Store':
        continue
        
    print(filing)
    matches = re.finditer('..>ITEM.{1,10}7[^a-zA-Z]{1,10}management', raw_10k, re.IGNORECASE)
    print([x for x in matches])
    
    matches = re.finditer('..>ITEM.{1,10}8(|A)[^a-zA-Z]{1,10}(consolidated|financial)', raw_10k, re.IGNORECASE)
    print([x for x in matches])
    print('\n')

0000004515-08-000014
[<re.Match object; span=(723646, 723672), match='/a>ITEM&#160;7. MANAGEMENT'>]
[<re.Match object; span=(1614421, 1614455), match='/a>ITEM&#160;8A.&#160;CONSOLIDATED'>]


0000006201-20-000023
[<re.Match object; span=(723646, 723672), match='/a>ITEM&#160;7. MANAGEMENT'>]
[<re.Match object; span=(1614421, 1614455), match='/a>ITEM&#160;8A.&#160;CONSOLIDATED'>]


0000950134-06-003715
[<re.Match object; span=(723646, 723672), match='/a>ITEM&#160;7. MANAGEMENT'>]
[<re.Match object; span=(1614421, 1614455), match='/a>ITEM&#160;8A.&#160;CONSOLIDATED'>]


0001047469-03-013301
[<re.Match object; span=(723646, 723672), match='/a>ITEM&#160;7. MANAGEMENT'>]
[<re.Match object; span=(1614421, 1614455), match='/a>ITEM&#160;8A.&#160;CONSOLIDATED'>]


0000950134-05-003726
[<re.Match object; span=(723646, 723672), match='/a>ITEM&#160;7. MANAGEMENT'>]
[<re.Match object; span=(1614421, 1614455), match='/a>ITEM&#160;8A.&#160;CONSOLIDATED'>]


0000950134-04-002668
[<re.Match object; span=

In [175]:
# Do the same for all files
companies = os.listdir("sec-edgar-filings/")
companies = [x for x in companies if x != 'AAL' and x != '.DS_Store']

for company in companies:
    
    logstr = ''
    
    if company == '.DS_Store' or company == 'AAL':
        continue
        
    pulls = os.listdir("sec-edgar-filings/"+company+"/10-K/")
    
    a = pd.DataFrame()
    
    
    for year in pulls:
        if year == '.DS_Store':
            continue
        #print(year)

        try:
            f = open("sec-edgar-filings/"+company+"/10-K/"+year+"/filing-details.html", "r")
        except:
            f = open("sec-edgar-filings/"+company+"/10-K/"+year+"/filing-details.txt", "r")
        raw_10k = f.read()
        
        matches1 = re.finditer(r"(?s).>IT.{0,20}EM.{1,20}7.{1,400}management", raw_10k, re.IGNORECASE)
        matches2 = re.finditer(r"(?s).>IT.{0,20}EM.{1,20}8(|A).{1,400}(consolidated|financial)", raw_10k, re.IGNORECASE)
        
        logstr += year + '\n'
        logstr += str([x for x in matches1])+ '\n'
        logstr += str([x for x in matches2])+ '\n'
        logstr += '\n'
        
    print(logstr, file=open("logs/"+company+".txt", "w"))

In [138]:
pulls

['0000912057-02-012933']

In [16]:
os.listdir("sec-edgar-filings/"+company+"/10-K/")

['0000912057-02-012933']

# Debug

In [181]:
company = '0001362468'
year = '0001047469-10-001935'

f = open("sec-edgar-filings/"+company+"/10-K/"+year+"/filing-details.html", "r")
raw_10k = str(f.read())

In [191]:
matches1 = re.finditer(r"(?s).IT.{0,20}EM.{1,20}7[^a].{1,400}", raw_10k, re.IGNORECASE)
a = [x for x in matches1]
print(a)

[<re.Match object; span=(23925, 24333), match='_item_7._management_s_discussio__ite03668"><p sty>, <re.Match object; span=(24644, 25052), match='_item_7._management_s_discussio__ite03668"><font >, <re.Match object; span=(372072, 372480), match='_item_7._management_s_discussio__ite03668"> </a>\>, <re.Match object; span=(576697, 577119), match='with the remaining 17 aircraft owned free and cle>]


In [189]:
locs = [x.start() for x in a]
locs

[260, 23925, 24644, 25095, 372072, 576697, 609081]

In [190]:
raw_10k[locs[0]-100:locs[0]+100]

'e document<br/>\n<a href="#bg41401_table_of_contents">  TABLE OF CONTENTS</a> <br/>\n<a href="#dm41401_item_7a._quantitative_and_qual__ite02669">  Item 7A. Quantitative and Qualitative Disclosures about'

In [180]:
matches2 = re.finditer('(?s).>IT.{0,20}EM.{1,20}8(|A).{1,400}FINANCIAL', raw_10k, re.IGNORECASE)
b = [x for x in matches2]
print(b)

[]


In [168]:
locs = [x.start() for x in b]
locs

[40459]

In [172]:
raw_10k[locs[0]-200:locs[0]+400]

'p style="margin-bottom:0pt;margin-top:0pt;margin-left:10pt;;text-indent:-10pt;;font-size:10pt;font-family:Times New Roman;font-weight:normal;font-style:normal;text-transform:none;font-variant: normal;">Item&#160;8.&#160;</p></td>\n<td style="padding-left:0pt;padding-Right:0pt;padding-Top:0pt;padding-Bottom:0pt;width:79.34%;" valign="bottom">\n<p style="margin-bottom:0pt;margin-top:0pt;margin-left:0pt;;text-indent:0pt;;font-size:10pt;font-family:Times New Roman;font-weight:normal;font-style:normal;text-transform:none;font-variant: normal;"><a href="#ITEM8_FINANCIAL_STATEMENTS"><font style="text-d'

In [242]:
# coding=utf8
# the above tag defines encoding for this document and is for Python 2.x compatibility

import re

regex = r".>ITEM.{1,20}7.*(?<=MANAGEMENT)"

test_str = "'font style=\"DISPLAY: inline; FONT-FAMILY: times new roman; FONT-SIZE: 10pt\">&#160;&#160;<a href=\"#mda\">ITEM&#160;7.</a></font></div>\\n</td>\\n<td align=\"left\" colspan=\"3\" valign=\"top\" width=\"83%\">\\n<div align=\"left\" style=\"TEXT-INDENT: 0pt; DISPLAY: block; MARGIN-LEFT: 18pt; MARGIN-RIGHT: 0pt\"><font style=\"DISPLAY: inline; FONT-FAMILY: times new roman; FONT-SIZE: 10pt\"><a href=\"#mda\">MANAGEMENT&#8217;'"

matches = re.finditer(regex, raw_10k, re.MULTILINE)

for matchNum, match in enumerate(matches, start=1):
    
    print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))
    
    for groupNum in range(0, len(match.groups())):
        groupNum = groupNum + 1
        
        print ("Group {groupNum} found at {start}-{end}: {group}".format(groupNum = groupNum, start = match.start(groupNum), end = match.end(groupNum), group = match.group(groupNum)))

# Note: for Python 2.7 compatibility, use ur"" to prefix the regex and u"" to prefix the test string and substitution.

# Go the Other Way

We have now built upwards, but some matches are too general and catch too many exceptions. Now, we must work backwards to find out why this is the case and see what we can do about it

In [207]:
# Do the same for all files
companies = os.listdir("sec-edgar-filings/")
companies = [x for x in companies if x != 'AAL' and x != '.DS_Store']

for company in companies:
    
    logstr = ''
    
    if company == '.DS_Store' or company == 'AAL':
        continue
        
    pulls = os.listdir("sec-edgar-filings/"+company+"/10-K/")
    
    a = pd.DataFrame()
    
    
    for year in pulls:
        if year == '.DS_Store':
            continue
        #print(year)

        try:
            f = open("sec-edgar-filings/"+company+"/10-K/"+year+"/filing-details.html", "r")
        except:
            continue
            #f = open("sec-edgar-filings/"+company+"/10-K/"+year+"/filing-details.txt", "r")
        raw_10k = f.read()
        
        matches1 = re.finditer(r"(?s).>IT.{0,20}EM.{1,20}7.{1,400}MANAGEMENT", raw_10k, re.IGNORECASE)
        matches2 = re.finditer(r"(?s).>IT.{0,20}EM.{1,20}8(|A).{1,400}(CONSOLIDATED|FINANCIAL)", raw_10k, re.IGNORECASE)
        
        list_matches1 = [x for x in matches1]
        list_matches2 = [x for x in matches2]
        
        logstr += year + '\n'
        logstr += str(list_matches1)+ '\n'
        logstr += str(list_matches2)+ '\n'
        logstr += '\n'
        
        if len(list_matches1) == 0:
            print('matches1 '+company+' '+year+'\n')
            
        if len(list_matches1) == 0:
            print('matches2 '+company+' '+year+'\n')
        
    #print(logstr, file=open("logs/"+company+".txt", "w"))

matches1 0001172222 0000950136-05-001827

matches2 0001172222 0000950136-05-001827

matches1 0001362468 0001047469-09-002190

matches2 0001362468 0001047469-09-002190

matches1 0001362468 0001047469-10-001935

matches2 0001362468 0001047469-10-001935



# Debug

In [196]:
company = '0000100517'
year = '0000100517-04-000007'

f = open("sec-edgar-filings/"+company+"/10-K/"+year+"/filing-details.html", "r")
raw_10k = str(f.read())

In [210]:
matches1 = re.finditer(r"(?s).IT.{0,20}EM.{1,20}7[^a].{1,400}MANAGEMENT", raw_10k, re.IGNORECASE)
a = [x for x in matches1]
print(a)

[<re.Match object; span=(9958, 10069), match='>ITEM 7.</font></td>\n<td width="1%"><font size=">, <re.Match object; span=(106868, 106911), match=' Item 7 </font><font size="2"><i>Management'>, <re.Match object; span=(144873, 145031), match='_item_7._management_s_discussio__ite03649"> </a>\>, <re.Match object; span=(616867, 616886), match='>ITEM 7. MANAGEMENT'>]


In [ ]:
## locs = [x.start() for x in a]
locs

In [219]:
idx = 3
raw_10k[locs[idx]-1000:locs[idx]+1000]

' size="2"><a href="#toc_bg1156_1">TABLE OF CONTENTS</a></font><br/>\n<!-- TOC_BEGIN -->\n<font size="2"><a href="#toc_de1156_1">FORWARD-LOOKING STATEMENTS</a></font><br/>\n<font size="2"><a href="#toc_de1156_2">PART I</a></font><br/>\n<ul>\n<font size="2"><a href="#toc_de1156_3">ITEM 1. BUSINESS.</a></font><br/>\n</ul>\n<!-- TOC_BEGIN -->\n<ul>\n<font size="2"><a href="#toc_dg1156_1">ITEM 2. PROPERTIES.</a></font><br/>\n<font size="2"><a href="#toc_dg1156_2">ITEM 4. SUBMISSION OF MATTERS TO A VOTE OF SECURITY HOLDERS.</a></font><br/>\n</ul>\n<font size="2"><a href="#toc_dg1156_3">PART II</a></font><br/>\n<ul>\n<font size="2"><a href="#toc_dg1156_4">ITEM 5. MARKET PRICE AND DIVIDEND INFORMATION.</a></font><br/>\n<font size="2"><a href="#toc_dg1156_5">ITEM 6. SELECTED FINANCIAL DATA.</a></font><br/>\n</ul>\n<font size="2"><a href="#toc_dg1156_6">Hawaiian Airlines, Inc. Selected Financial and Statistical Data (in thousands, except per share data)</a></font><br/>\n<ul>\n<font size="2"><a 

In [180]:
matches2 = re.finditer('(?s).>IT.{0,20}EM.{1,20}8(|A).{1,400}FINANCIAL', raw_10k, re.IGNORECASE)
b = [x for x in matches2]
print(b)

[]


In [168]:
locs = [x.start() for x in b]
locs

[40459]

In [172]:
raw_10k[locs[0]-200:locs[0]+400]

'p style="margin-bottom:0pt;margin-top:0pt;margin-left:10pt;;text-indent:-10pt;;font-size:10pt;font-family:Times New Roman;font-weight:normal;font-style:normal;text-transform:none;font-variant: normal;">Item&#160;8.&#160;</p></td>\n<td style="padding-left:0pt;padding-Right:0pt;padding-Top:0pt;padding-Bottom:0pt;width:79.34%;" valign="bottom">\n<p style="margin-bottom:0pt;margin-top:0pt;margin-left:0pt;;text-indent:0pt;;font-size:10pt;font-family:Times New Roman;font-weight:normal;font-style:normal;text-transform:none;font-variant: normal;"><a href="#ITEM8_FINANCIAL_STATEMENTS"><font style="text-d'